In [2]:
import skimage.io as io
import numpy as np
from skimage.filters import threshold_otsu 
from skimage.color import rgb2grey
from skimage.measure import find_contours
import cv2
from windows import getClusteredWindows

In [3]:
def First_difference(chain_code):
    #chain =chain_code[-1] +chain_code
    chain = chain_code
    chain = list(chain)
    chain_shifted = chain[1:]
    chain = chain[:-1]
    chain = np.array(chain).astype(np.int)
    chain_shifted = np.array(chain_shifted).astype(np.int)
    first_diff = -1*(chain - chain_shifted)%8
    first_diff = first_diff.astype(np.str)
    return "".join(first_diff)
    

In [62]:
def Chain_Codes_Counting(contours,pairs_triplets=True):
    global_chaining_code = ""
    first_difference = ""
    second_difference = ""
    if(pairs_triplets):
        F4 = np.zeros(64)
        F5 = np.zeros(512)
    
    for j in range(len(contours)):
        c = contours[j]
        local_chaining_code = ""
        for i in range(1,len(c)):
            diff = (c[i]-c[i-1])[0]
            if(diff[0]==1 and diff[1]==0):
                local_chaining_code +="0"
            elif(diff[0]==1 and diff[1]==-1):
                local_chaining_code+="1"
            elif(diff[0]==0 and diff[1]==-1):
                local_chaining_code +="2"
            elif(diff[0]==-1 and diff[1]==-1):
                local_chaining_code+="3"
            elif(diff[0]==-1 and diff[1]==0):
                local_chaining_code+="4"
            elif(diff[0]==-1 and diff[1]==1):
                local_chaining_code +="5"
            elif(diff[0]==0 and diff[1]==1):
                local_chaining_code+="6"
            elif(diff[0]==1 and diff[1]==1):
                local_chaining_code+="7"
            else:
                print("ERROR")

        if(len(local_chaining_code)>0):
            first_diff = First_difference(local_chaining_code)
            first_difference += first_diff    
            second_difference += First_difference(first_diff)
        global_chaining_code+=local_chaining_code
        if pairs_triplets:
            for  i in range(8):
                for j in range(8):
                    check = str(i)+str(j)
                    F4[i*8+j] += local_chaining_code.count(check)
                    for k in range(8):
                        check = str(i)+str(j)+str(k)
                        F5[i*64+j*8+k] += local_chaining_code.count(check)
    F1  = []
    F2  = []
    F3  = []
    for i in range(8):
        F1.append(global_chaining_code.count(str(i)))
        if i!=5:
            F2.append(first_difference.count(str(i)))

        F3.append(second_difference.count(str(i)))

    F1 = np.array(F1)
    F2 = np.array(F2)
    F3 = np.array(F3)
    F1 = F1/F1.sum()
    F2 = F2/F2.sum()
    F3 = F3/F3.sum()
    if pairs_triplets:
        F4 = F4/F4.sum()
        F5 = F5/F5.sum()
        return F1,F2,F3,F4,F5
    return F1,F2,F3

In [65]:
def Chain_Codes_Feature_Extractor(image_name):
    img = io.imread(image_name)
    windows, clusters, clusteredWindows = getClusteredWindows(img,13,10)
   
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    
    #img = np.ones(img.shape)*255
    #cv2.drawContours(img, contours, -1, (30, 255, 30), 1) 
    
    F1,F2,F3,F4,F5 = Chain_Codes_Counting(contours,True)
    
    F7 = np.array([])
    F8 = np.array([])
    F9 = np.array([])
    
    for i in range(clusteredWindows.shape[0]):
        edged = cv2.Canny(clusteredWindows[i].astype(np.uint8), 30, 200)
        contours, hierarchy = cv2.findContours(edged,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        F7_local , F8_local , F9_local  = Chain_Codes_Counting(contours,False)
        F7 = np.append(F7_local,F7)
        F8 = np.append(F8_local,F8)
        F9 = np.append(F9_local,F9)
    
    
    
    
    return F1,F2,F3,F4,F5,F7,F8,F9
    
    




   

    
    

In [66]:
F1,F2,F3,F4,F5,F7,F8,F9 = Chain_Codes_Feature_Extractor('TestCases/0.png')

In [69]:
print(F8,len(F8))

[0.32608696 0.2173913  0.02173913 0.02173913 0.08695652 0.04347826
 0.2826087  0.42857143 0.24489796 0.         0.02040816 0.04081633
 0.         0.26530612 0.14705882 0.35294118 0.02941176 0.
 0.08823529 0.02941176 0.35294118 0.2173913  0.39130435 0.
 0.04347826 0.         0.         0.34782609 0.18604651 0.1627907
 0.06976744 0.09302326 0.06976744 0.11627907 0.30232558 0.65909091
 0.13636364 0.         0.02272727 0.04545455 0.         0.13636364
 0.95454545 0.         0.         0.         0.04545455 0.
 0.         0.72727273 0.09090909 0.02272727 0.         0.04545455
 0.02272727 0.09090909 0.54347826 0.2173913  0.         0.
 0.04347826 0.04347826 0.15217391 0.40740741 0.27777778 0.01851852
 0.         0.03703704 0.03703704 0.22222222] 70
